In [2]:
### 📚 **Importar Librerías**
import pandas as pd
import numpy as np
import cv2 as cv
import os
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# ✅ **Configuración Global**
IMG_SIZE = (96, 96)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# ✅ **Rutas de Datos**
ruta_train_csv = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train.csv'
ruta_test_csv = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test.csv'
ruta_train_img = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train'
ruta_test_img = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'

# ✅ **Cargar CSV**
df_train = pd.read_csv(ruta_train_csv)
df_test = pd.read_csv(ruta_test_csv)

### 🛠️ **Preprocesamiento de Imágenes**
def preprocesar_imagen(img_path):
    img = cv.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {img_path}")
    b, _, _ = cv.split(img)
    blur = cv.GaussianBlur(b, (3, 3), cv.BORDER_DEFAULT)
    _, thresh = cv.threshold(blur, 225, 255, cv.THRESH_BINARY)
    thresh = 255 - thresh
    canny = cv.Canny(thresh, 0, 25)
    dilated = cv.dilate(canny, (15, 15), iterations=2)
    eroded = cv.erode(dilated, (7, 7), iterations=1)
    resized_img = cv.resize(eroded, IMG_SIZE)
    return resized_img / 255.0

# ✅ **Procesar Imágenes en Carpeta**
def procesar_imagenes(df, ruta_img):
    imagenes = []
    for id in tqdm(df['id'], desc='Procesando imágenes'):
        img_path = os.path.join(ruta_img, f'clips-{id}.png')
        imagen = preprocesar_imagen(img_path)
        imagenes.append(imagen)
    return np.array(imagenes)

# ✅ **Cargar Datos de Entrenamiento y Prueba**
X_train = procesar_imagenes(df_train, ruta_train_img)
X_test = procesar_imagenes(df_test, ruta_test_img)

y_train = df_train['clip_count'].astype('float32').values

# ✅ **Dividir Datos en Entrenamiento y Validación**
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Añadir dimensión para canales (escala de grises)
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

### 🤖 **Definir Modelo CNN**
modelo = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1)  # Salida para regresión
])

modelo.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae']
)

modelo.summary()

### ✅ **Callbacks**
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

### 🚀 **Entrenar Modelo**
historial = modelo.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=BATCH_SIZE,
    callbacks=[callback_early_stopping, callback_reduce_lr]
)

### 📊 **Evaluación del Modelo**
test_loss, test_mae = modelo.evaluate(X_val, y_val)
print(f'Test MAE: {test_mae}')

### 📈 **Visualizar Resultados**
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(historial.history['loss'], label='Pérdida Entrenamiento')
plt.plot(historial.history['val_loss'], label='Pérdida Validación')
plt.legend()
plt.title('Pérdida')

plt.subplot(1, 2, 2)
plt.plot(historial.history['mae'], label='MAE Entrenamiento')
plt.plot(historial.history['val_mae'], label='MAE Validación')
plt.legend()
plt.title('Error Absoluto Medio')
plt.show()

### 📝 **Predicciones en Datos de Prueba**
predicciones = modelo.predict(X_test)

# ✅ **Guardar Predicciones**
df_test['clip_count'] = predicciones.flatten()
df_test.to_csv('resultados_test.csv', index=False)
print('✅ Predicciones guardadas en resultados_test.csv')

### 🎯 **Fin del Proceso**
print('¡Todo el proceso ha finalizado correctamente!')


Procesando imágenes: 100%|██████████| 5000/5000 [01:09<00:00, 71.64it/s]
c:\Users\mikel\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 94, 94, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 47, 47, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 45, 45, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,965,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,984,001 (15.20 MB)

 Trainable params: 3,984,001 (15.20 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 36s 93ms/step - loss: 157.6286 - mae: 8.5380 - val_loss: 19.5528 - val_mae: 3.3575 - learning_rate: 0.0010
Epoch 2/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 36s 95ms/step - loss: 52.9984 - mae: 5.2888 - val_loss: 13.0458 - val_mae: 2.6817 - learning_rate: 0.0010
Epoch 3/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 40s 107ms/step - loss: 49.1970 - mae: 5.0336 - val_loss: 26.1281 - val_mae: 3.8516 - learning_rate: 0.0010
Epoch 4/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 35s 93ms/step - loss: 52.5145 - mae: 5.1454 - val_loss: 22.6243 - val_mae: 3.4818 - learning_rate: 0.0010
Epoch 5/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 34s 91ms/step - loss: 46.3542 - mae: 4.8211 - val_loss: 10.9787 - val_mae: 2.4272 - learning_rate: 0.0010
Epoch 6/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 34s 91ms/step - loss: 46.0200 - mae: 4.8554 - val_loss: 10.8272 - val_mae: 2.4252 - learning_rate: 0.0010
Epoch 7/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 35s 93ms/step - loss: 44.4811 - mae: 4.7502 - val_loss: 11.9260 - val_mae: 2.5283 

KeyboardInterrupt: 